In [1]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import glob
import os
from mpl_toolkits.mplot3d import Axes3D
import SimpleITK as sitk
import myshow

In [6]:
def view_stats():
    pdata = pd.DataFrame()
    for metafile_path in glob.glob('../datasets/meta/*.csv'):
        newdata = pd.read_csv(metafile_path, index_col=0)
        pdata = pdata.append(newdata, ignore_index=True)
    
    # Restrict only to brats for now
    pdata = pdata[pdata["dataset_name"]=='BRATS2015']
    
    fig = plt.figure()
    ax = Axes3D(fig)
    colors = pd.Categorical(pdata["dataset_name"]).codes
    ax.scatter(pdata['x_spacing'], pdata['y_spacing'], pdata['z_spacing'], c=colors, linewidth=0.1)
    ax.name = "Spacing"
    ax.set_xlabel("x_spacing")
    ax.set_ylabel("y_spacing")
    ax.set_zlabel("z_spacing")
    ax.set_title("Spacing (voxel size in mm)")
    plt.show()
    
    fig = plt.figure()
    ax = Axes3D(fig)
    colors = pd.Categorical(pdata["dataset_name"]).codes
    ax.scatter(pdata['x_dimesnion'], pdata['y_dimesnion'], pdata['z_dimension'], c=colors, linewidth=0.1)
    ax.name = "Spacing"
    ax.set_xlabel("x_dimension")
    ax.set_ylabel("y_dimension")
    ax.set_zlabel("z_dimension")
    ax.set_title("Number of voxels")
    plt.show()
    
    fig = plt.figure()
    ax = Axes3D(fig)
    colors = pd.Categorical(pdata["dataset_name"]).codes
    ax.scatter(pdata['x_origin'], pdata['y_origin'], pdata['z_origin'], c=colors, linewidth=0.1)
    ax.name = "Origin"
    ax.set_xlabel("x_origin")
    ax.set_ylabel("y_origin")
    ax.set_zlabel("z_origin")
    ax.set_title("Origin position")
    plt.show()
    
    print(pdata[pdata["y_origin"]!=0])
    print(pdata[pdata["y_origin"]==0])
    


In [3]:
import SimpleITK as sitk
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive
from ipywidgets import widgets

def myshow(img, title=None, margin=0.05, dpi=80 ):
    nda = sitk.GetArrayFromImage(img)

    spacing = img.GetSpacing()
    slicer = False

    if nda.ndim == 3:
        # fastest dim, either component or x
        c = nda.shape[-1]

        # the the number of components is 3 or 4 consider it an RGB image
        if not c in (3,4):
            slicer = True

    elif nda.ndim == 4:
        c = nda.shape[-1]

        if not c in (3,4):
            raise Runtime("Unable to show 3D-vector Image")

        # take a z-slice
        slicer = True

    if (slicer):
        ysize = nda.shape[1]
        xsize = nda.shape[2]
    else:
        ysize = nda.shape[0]
        xsize = nda.shape[1]


    # Make a figure big enough to accomodate an axis of xpixels by ypixels
    # as well as the ticklabels, etc...
    figsize = (1 + margin) * ysize / dpi, (1 + margin) * xsize / dpi
    def callback(z=None):

        extent = (0, xsize*spacing[1], ysize*spacing[0], 0)

        fig = plt.figure(figsize=figsize, dpi=dpi)

        # Make the axis the right size...
        ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])

        plt.set_cmap("gray")

        if z is None:
            ax.imshow(nda,extent=extent,interpolation=None)
        else:
            ax.imshow(nda[z,...],extent=extent,interpolation=None)

        if title:
            plt.title(title)

        plt.show()

    if slicer:
        interact(callback, z=(0,nda.shape[0]-1))
    else:
        callback()

def myshow3d(img, xslices=[], yslices=[], zslices=[], title=None, margin=0.05, dpi=80):
    size = img.GetSize()
    img_xslices = [img[s,:,:] for s in xslices]
    img_yslices = [img[:,s,:] for s in yslices]
    img_zslices = [img[:,:,s] for s in zslices]

    maxlen = max(len(img_xslices), len(img_yslices), len(img_zslices))


    img_null = sitk.Image([0,0], img.GetPixelID(), img.GetNumberOfComponentsPerPixel())

    img_slices = []
    d = 0

    if len(img_xslices):
        img_slices += img_xslices + [img_null]*(maxlen-len(img_xslices))
        d += 1

    if len(img_yslices):
        img_slices += img_yslices + [img_null]*(maxlen-len(img_yslices))
        d += 1

    if len(img_zslices):
        img_slices += img_zslices + [img_null]*(maxlen-len(img_zslices))
        d +=1

    if maxlen != 0:
        if img.GetNumberOfComponentsPerPixel() == 1:
            img = sitk.Tile(img_slices, [maxlen,d])
        #TODO check in code to get Tile Filter working with VectorImages
        else:
            img_comps = []
            for i in range(0,img.GetNumberOfComponentsPerPixel()):
                img_slices_c = [sitk.VectorIndexSelectionCast(s, i) for s in img_slices]
                img_comps.append(sitk.Tile(img_slices_c, [maxlen,d]))
            img = sitk.Compose(img_comps)

In [17]:
def show_mri():
    pdata = pd.DataFrame()
    for metafile_path in glob.glob('../datasets/meta/*.csv'):
        newdata = pd.read_csv(metafile_path, index_col=0)
        pdata = pdata.append(newdata, ignore_index=True)
    
    # Restrict only to brats for now
    pdata = pdata[pdata["dataset_name"]=='BRATS2015']
    
    print(pdata[pdata["y_origin"]!=0]['mri_type'])
    print(pdata[pdata["y_origin"]==0]['mri_type'])
    
    origin_nz = pdata[pdata["y_origin"]!=0].sample()["path"].values[0]
    origin_z = pdata[pdata["y_origin"]==0].sample()["path"].values[0]
    img = myshow(sitk.ReadImage(origin_nz))
    img = myshow(sitk.ReadImage(origin_z))
    
show_mri()

1428    MR_Flair
1429       MR_T1
1430      MR_T1c
1431       MR_T2
1432          OT
1433    MR_Flair
1434       MR_T1
1435      MR_T1c
1436       MR_T2
1437          OT
1438    MR_Flair
1439       MR_T1
1440      MR_T1c
1441       MR_T2
1442          OT
1443    MR_Flair
1444       MR_T1
1445      MR_T1c
1446       MR_T2
1447          OT
1448    MR_Flair
1449       MR_T1
1450      MR_T1c
1451       MR_T2
1452          OT
1453    MR_Flair
1454       MR_T1
1455      MR_T1c
1456       MR_T2
1457          OT
          ...   
2760      MR_T1c
2761       MR_T2
2763    MR_Flair
2764       MR_T1
2765      MR_T1c
2766       MR_T2
2768    MR_Flair
2769       MR_T1
2770      MR_T1c
2771       MR_T2
2773    MR_Flair
2774       MR_T1
2775      MR_T1c
2776       MR_T2
2778    MR_Flair
2779       MR_T1
2780      MR_T1c
2781       MR_T2
2783    MR_Flair
2784       MR_T1
2785      MR_T1c
2786       MR_T2
2788    MR_Flair
2789       MR_T1
2790      MR_T1c
2791       MR_T2
2793    MR_Flair
2794       MR_

interactive(children=(IntSlider(value=77, description='z', max=154), Output()), _dom_classes=('widget-interact…

interactive(children=(IntSlider(value=77, description='z', max=154), Output()), _dom_classes=('widget-interact…